# Deduplication using LSH
- input: 
    - data3: dataframe with 'text' english columns
- output: 
    - data4: dataframe without similar articles, based on 'text' columns

In [2]:
import pandas
data3=pandas.read_csv('../data/4.Tweet data after clear spams.csv')
data3.tail()

,Unnamed: 0,index,created_at,id,text,language
219,227,251.0,Tue Apr 27 23:58:10 +0000 2021,1.387194e+18,Sometimes you get a third chance Love s Thir...,en
220,228,252.0,Tue Apr 27 23:58:10 +0000 2021,1.387194e+18,Check out this Amazon deal The Art of My Neig...,en
221,229,253.0,Tue Apr 27 23:58:08 +0000 2021,1.387194e+18,First Steps How Upright Walking Made Us Hum...,en
222,230,254.0,Tue Apr 27 23:58:04 +0000 2021,1.387194e+18,The First Day of Spring by Nancy Tucker B...,en
223,231,255.0,Tue Apr 27 23:58:04 +0000 2021,1.387194e+18,ad off Galaxy Star Projector ...,en


In [3]:
import os
import re
import random
import time
import binascii
from bisect import bisect_right
from heapq import heappop, heappush
import sys
from __future__ import division

In [4]:
# Convert documents to sets of shingles (shingle=3 consecutive words in documents)

data3['shingleSet']=''
totalShingles = 0
numDocs=len(data3)
for i in range(0, numDocs):
    words = str(data3.text[i]).split(" ") 
    shingleSet = set()
    for index in range(0, len(words) - 2):
        shingle = words[index] + " " + words[index + 1] + " " + words[index + 2]
        crc = binascii.crc32(shingle.encode()) & 0xffffffff # Hash the shingle to a 32-bit integer
        shingleSet.add(crc)
    data3['shingleSet'][i] = shingleSet
    totalShingles = totalShingles + len(shingleSet)

data3.tail()

ipykernel_launcher:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Unnamed: 0,index,created_at,id,text,language,shingleSet
219,227,251.0,Tue Apr 27 23:58:10 +0000 2021,1.387194e+18,Sometimes you get a third chance Love s Thir...,en,"{1975646848, 3508350977, 2277555713, 187718236..."
220,228,252.0,Tue Apr 27 23:58:10 +0000 2021,1.387194e+18,Check out this Amazon deal The Art of My Neig...,en,"{622155522, 3082850439, 1706091273, 1593029644..."
221,229,253.0,Tue Apr 27 23:58:08 +0000 2021,1.387194e+18,First Steps How Upright Walking Made Us Hum...,en,"{2352757504, 1318699267, 463207690, 4013102741..."
222,230,254.0,Tue Apr 27 23:58:04 +0000 2021,1.387194e+18,The First Day of Spring by Nancy Tucker B...,en,"{3712246153, 2391777162, 2684049684, 401310274..."
223,231,255.0,Tue Apr 27 23:58:04 +0000 2021,1.387194e+18,ad off Galaxy Star Projector ...,en,"{2473481, 4013102741, 566037670, 2455870758, 1..."


In [5]:
totalShingles /numDocs #Average shingles per doc

21.330357142857142

In [6]:
# When comparing 2 documents, instead of compairing all shingles, we only compair random 30 shingles, by converting each document to a signature of length 30
numHashes = 10 #The length of signature designed for each document, make a justment based on average shingles per doc

In [7]:
numElems = int(numDocs * (numDocs - 1) / 2) # number of paired documents to compare similarities
estJSim = [0 for x in range(numElems)] #'estJSim' will be for the estimated Jaccard Similarities found by comparing the MinHash signatures.

# Define a function to map a 2D matrix coordinate into a 1D index.
def getTriangleIndex(i, j):
    if i == j:
        sys.stderr.write("Can't access triangle matrix with i == j")
        sys.exit(1)
    if j < i:
        temp = i
        i = j
        j = temp
    return int(i * (numDocs - (i + 1) / 2.0) + j - i) - 1

In [8]:
maxShingleID = 2**32-1
def pickRandomCoeffs(k):  # Create a list of 'k' random values between 0 and maxShingleID
    randList = []
    while k > 0:
        randIndex = random.randint(0, maxShingleID) 
        while randIndex in randList:
            randIndex = random.randint(0, maxShingleID) 
        randList.append(randIndex)
        k = k - 1
    return randList

# Generating random hash functions

nextPrime = 4294967311 # The prime number above 'maxShingleID'

coeffA,coeffB = pickRandomCoeffs(numHashes),pickRandomCoeffs(numHashes) #each pair corresponse to a function that creat the first element in each signature

# Generating MinHash signatures for all documents

data3['signature'] = ''

for index in range(0,numDocs):
    shingleSet = data3['shingleSet'][index]
    signature = []
    for i in range(0, numHashes): #creating each elemment of a signature
        minHashCode = nextPrime + 1
        for shingle in shingleSet:
            hashCode = (coeffA[i] * shingle + coeffB[i]) % nextPrime 
            if hashCode < minHashCode:# Track the lowest hash code seen
                minHashCode = hashCode
        signature.append(minHashCode)
    data3['signature'][index] = signature

data3.head()

ipykernel_launcher:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Unnamed: 0,index,created_at,id,text,language,shingleSet,signature
0,0,0.0,Sun May 02 07:59:00 +0000 2021,1.388765e+18,House of Cards has always hit differently ...,en,"{3455063171, 1408802948, 3967006724, 483391110...","[69277892, 29551887, 9488767, 21443863, 130321..."
1,2,2.0,Sun May 02 14:21:00 +0000 2021,1.388861e+18,Select st party Switch physical digita...,en,"{3481900867, 2620196452, 237583687, 1968184171...","[357313354, 200396749, 330873488, 80418240, 10..."
2,3,3.0,Mon May 03 23:59:58 +0000 2021,1.389369e+18,Silicone Eye Pad for Oculus Quest Face Cushi...,en,"{594769416, 2740734737, 2450043928, 3640267417...","[58480877, 138627901, 24612090, 796973078, 487..."
3,4,4.0,Mon May 03 23:59:52 +0000 2021,1.389369e+18,Power Strip Surge Protector Outlets USB P...,en,"{693199104, 2065756037, 369095176, 626586633, ...","[63015609, 17752544, 1560977, 50567359, 389340..."
4,6,6.0,Mon May 03 23:59:43 +0000 2021,1.389369e+18,Hi You can turn off the microphone When thi...,en,"{1538276102, 1610259334, 890091017, 3614318604...","[209003528, 69077452, 49794868, 67688133, 5626..."


In [9]:
# Generate pairs of duplicate articles
threshold = 0.5  

docA = []  #first document in the duplicate pair - store uuid here
docB = []  #second document in the duplicate pair - store uuid here
for i in range(0, numDocs):  
    for j in range(i + 1, numDocs):
        estJ = estJSim[getTriangleIndex(i, j)]
        if estJ > threshold:
            docA.append(i)
            docB.append(j)
    
    #   Calculate the actual Jaccard similarity for validation. Comment out this part when you done checking
            # s1 = data3['shingleSet'][i]
            # s2 = data3['shingleSet'][j]
            # if len(s1.union(s2)) != 0:
            #     J = (len(s1.intersection(s2)) / len(s1.union(s2)))
            # print ("  %5s --> %5s   %.2f     %.2f" % (i, j, estJ, J))

In [10]:
# data3.loc[docA].text.to_csv('docA.csv')
# data3.loc[docB].text.to_csv('docB.csv')

In [12]:
# Remove duplicate documents
docs_to_remove = set(docB)
data4=data3.drop(docB)
data4.to_csv('../data/5.pulledTweet-deduplicated.csv')
data4.tail()

,Unnamed: 0,index,created_at,id,text,language,shingleSet,signature
219,227,251.0,Tue Apr 27 23:58:10 +0000 2021,1.387194e+18,Sometimes you get a third chance Love s Thir...,en,"{1975646848, 3508350977, 2277555713, 187718236...","[18622085, 32776440, 21561465, 191860621, 9160..."
220,228,252.0,Tue Apr 27 23:58:10 +0000 2021,1.387194e+18,Check out this Amazon deal The Art of My Neig...,en,"{622155522, 3082850439, 1706091273, 1593029644...","[116119196, 134412392, 15045377, 922378, 36390..."
221,229,253.0,Tue Apr 27 23:58:08 +0000 2021,1.387194e+18,First Steps How Upright Walking Made Us Hum...,en,"{2352757504, 1318699267, 463207690, 4013102741...","[77244016, 247586965, 343106086, 653196371, 20..."
222,230,254.0,Tue Apr 27 23:58:04 +0000 2021,1.387194e+18,The First Day of Spring by Nancy Tucker B...,en,"{3712246153, 2391777162, 2684049684, 401310274...","[8379671, 18718807, 281343041, 177753045, 3669..."
223,231,255.0,Tue Apr 27 23:58:04 +0000 2021,1.387194e+18,ad off Galaxy Star Projector ...,en,"{2473481, 4013102741, 566037670, 2455870758, 1...","[144950540, 122747773, 41607353, 97773221, 829..."
